## 反向传播和正向传播

得到一个batch的前向传播结果后，需要定义一个损失函数来刻画当前的预测值和真实答案之间的误差。然后通过反向传播算法调整神级网络参数的取值，使差距可以被缩小。

In [1]:
import tensorflow as tf
from numpy.random import RandomState

In [5]:
batch_size = 8

w1 = tf.Variable(tf.random_normal([2,3],stddev=1,seed=1))
w2 = tf.Variable(tf.random_normal([3,1],stddev=1,seed=1))

#将数据分成batch
x = tf.placeholder(tf.float32,shape=(None,2),name='x-input')
y_=tf.placeholder(tf.float32,shape=(None,1),name='y-input')

#定义前向传播的过程
a = tf.matmul(x,w1)
y = tf.matmul(a,w2)

#定义损失函数和反向传播的算法
y = tf.sigmoid(y)
cross_entropy = -tf.reduce_mean(y_*tf.log(tf.clip_by_value(y,1e-10,1.0)) 
                                + (1-y)*tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)
#通过随机数生产模拟数据
rdm = RandomState(1)
dataset_size = 128
X= rdm.rand(dataset_size,2)

Y = [[int(x1+x2 < 1)] for (x1,x2) in X]

#session,训练之前数据
with tf.Session() as sess:
    init_op =tf.global_variables_initializer()
    
    sess.run(init_op)
    
    print(sess.run(w1))
    print(sess.run(w2))
    
    #设定轮数
    STEPS = 5000
    for i in range(STEPS):
        #每次选取batch_size
        start = (i* batch_size) % dataset_size
        end = min(start+batch_size,dataset_size)
        
        sess.run(train_step,feed_dict={x:X[start:end],y_:Y[start:end]})
        
        if i % 1000 == 0:
            total_crpss_entropy = sess.run(
                cross_entropy,feed_dict={x:X,y_:Y}
            )
            print ('After %d training step(s),cross entropy on all data is %g'%(i,total_crpss_entropy))
            
    print (sess.run(w1))
    print (sess.run(w2))

[[-0.8113182   1.4845988   0.06532937]
 [-2.4427042   0.09924842  0.5912244 ]]
[[-0.8113182 ]
 [ 1.4845988 ]
 [ 0.06532937]]
After 0 training step(s),cross entropy on all data is 0.314006
After 1000 training step(s),cross entropy on all data is 0.0684551
After 2000 training step(s),cross entropy on all data is 0.033715
After 3000 training step(s),cross entropy on all data is 0.020558
After 4000 training step(s),cross entropy on all data is 0.0136867
[[-2.548655   3.0793087  2.8951712]
 [-4.1112742  1.6259067  3.3972702]]
[[-2.3230937]
 [ 3.3011684]
 [ 2.4632084]]


## 步骤
1. 定义神经网络的结构和前向传播的输出结果
2. 定义损失函数以及选择反向传播的优化算法
3. 生成会话，并且在训练集上反复运行反向传播算法。

> 无论神经网络的结构如何变化，这三个步骤不会改变